In [6]:
import os
import ipywidgets as widgets
import plotly.graph_objs as go
import yfinance as yf
from IPython.display import display

In [7]:
def clean_str(string):
    new_str = ""
    for letter in string:
        if letter.isupper():
            new_str += " "
        new_str += letter
    return new_str.title()

def format_plotly(fig, x, y, ticker, expiry, inst):
    fig.update_yaxes(title=clean_str(x))
    fig.update_xaxes(title=clean_str(y))
    title = f"{clean_str(y)} vs. {clean_str(x)} for {ticker.upper()} {inst}s on {expiry}"
    fig.update_layout(
        autosize=False,
        width=1000,
        height=500,
        title={
            "text": title,
            "y": 0.95,
            "x": 0.5,
            "xanchor": "center",
            "yanchor": "top",
        },
    )

In [8]:
def view_chart(ticker, expiry, inst, x, y):
    global last_ticker, last_exp
    # set_interval_opts(interval_widget,start,end)
    if ticker:
        stock = yf.Ticker(ticker)
        if ticker != last_ticker:
            exps = stock.options
            exps = [""] + list(exps)
            exp_w.options = exps
            last_ticker = ticker

        if expiry:
            chains = stock.option_chain(expiry)
            df = chains.calls if inst == "Call" else chains.puts

            fig = go.Figure()
            plot = go.Scatter(
                x=df[x],
                y=df[y],
                mode="lines",
            )
            fig.add_trace(plot)

            format_plotly(fig, x, y, ticker, expiry, inst)

            if os.environ.get("SERVER_SOFTWARE", "jupyter").startswith("voila"):
                fig.show(config={"showTips": False}, renderer="notebook")
            else:
                fig.show(config={"showTips": False})


last_ticker = ""
last_exp = ""
opts = [
    "lastTradeDate",
    "strike",
    "lastPrice",
    "bid",
    "ask",
    "change",
    "percentChange",
    "volume",
    "openInterest",
    "impliedVolatility",
]

x_w = widgets.Dropdown(options=opts, value="strike", description="X")
y_w = widgets.Dropdown(options=opts, value="lastPrice", description="Y")
tickers_w = widgets.Text(description="Ticker")
exp_w = widgets.Dropdown(description="Expiry")
inst_w = widgets.Dropdown(options=["Put", "Call"], value="Call", description="Type")
controls = widgets.VBox([tickers_w, exp_w, inst_w, x_w, y_w])

stocks_view = widgets.interactive_output(
    view_chart,
    {"ticker": tickers_w, "expiry": exp_w, "inst": inst_w, "x": x_w, "y": y_w},
)

title_html = "<h1>Option Chain Dashboard</h1>"
app_contents = [
    widgets.HTML(title_html),
    controls,
    stocks_view,
]
app = widgets.VBox(app_contents)
display(app)
